setup

In [24]:
import json, glob, os
import tensorflow as tf

In [25]:
def read_json(path):
    return json.load(open(path))

def get_subpaths(path):
    return glob.glob(path)

In [26]:
ROOT_DIR = os.path.dirname(os.getcwd()) + "/"
ROOT_DIR

'/Users/user/juhochoi/'

In [27]:
# DATA_DIR = ROOT_DIR + "data/criteo/" # criteo
# RECORD_DIR = "data/tfrecord/" # criteo
DATA_DIR = ROOT_DIR + "data/place/" # place
RECORD_DIR = "data/" # place
DATA_DIR

'/Users/user/juhochoi/data/place/'

In [28]:
TARGET_DIR = DATA_DIR + "sample/"
TARGET_DIR

'/Users/user/juhochoi/data/place/sample/'

In [29]:
PART_DIRS = get_subpaths(TARGET_DIR + "part*/")
PART_DIRS

['/Users/user/juhochoi/data/place/sample/part01/',
 '/Users/user/juhochoi/data/place/sample/part00/']

In [30]:
# test ipynb with only one part
PART_DIR = PART_DIRS[0]
# should be:
# for PART_DIR in PART_DIRS 
# to run all partitions
PART_DIR

'/Users/user/juhochoi/data/place/sample/part01/'

In [31]:
filenames = get_subpaths(PART_DIR + RECORD_DIR + "part*")
filenames

['/Users/user/juhochoi/data/place/sample/part01/data/part-r-00000']

In [32]:
dataset = tf.data.TFRecordDataset(filenames = filenames)
dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [33]:
def filter_fn(x):
    nulls = tf.math.is_nan(x)
    print(nulls)
    return tf.reduce_all(nulls).eval()

In [42]:
for raw_record in dataset.take(10):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

features {
  feature {
    key: "ad_adgrp_id"
    value {
      bytes_list {
        value: "grp-m001-01-000000171983944"
      }
    }
  }
  feature {
    key: "ad_clk"
    value {
      float_list {
        value: 86.0
      }
    }
  }
  feature {
    key: "ad_coec"
    value {
      float_list {
        value: 1.934905767440796
      }
    }
  }
  feature {
    key: "ad_ctr"
    value {
      float_list {
        value: 0.02293333411216736
      }
    }
  }
  feature {
    key: "ad_cust_id"
    value {
      bytes_list {
        value: "373392"
      }
    }
  }
  feature {
    key: "ad_imp"
    value {
      float_list {
        value: 3750.0
      }
    }
  }
  feature {
    key: "ad_type"
    value {
      bytes_list {
        value: "1"
      }
    }
  }
  feature {
    key: "adxkwd_clk"
    value {
      float_list {
        value: 6.0
      }
    }
  }
  feature {
    key: "adxkwd_coec"
    value {
      float_list {
        value: 0.3693796694278717
      }
    }
  }
  featu

In [35]:
features = read_json(DATA_DIR + "features.json")
features

{'fields': [{'name': 'ncc_ad_id',
   'len': 1,
   'type': 'string',
   'default_value': ''},
  {'name': 'keyword', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'ncc_keyword_id', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'channel_id', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'sch_keyword', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'gender', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'age', 'len': 1, 'type': 'string', 'default_value': ''},
  {'name': 'clicked', 'len': 1, 'type': 'int64', 'default_value': 0},
  {'name': 'position_ctr', 'len': 1, 'type': 'float32', 'default_value': 0.0},
  {'name': 'chnlxposition_ctr',
   'len': 1,
   'type': 'float32',
   'default_value': 0.0},
  {'name': 'ad_imp', 'len': 1, 'type': 'float32', 'default_value': 0.0},
  {'name': 'ad_clk', 'len': 1, 'type': 'float32', 'default_value': 0.0},
  {'name': 'ad_ctr', 'len': 1, 'type': 'float32', 'default_value': 0.0},
  {'n

In [36]:
def parse(proto_bytes):
    feature_description = {}
    for feature in features["fields"]:
        feature_description[feature["name"]] = tf.io.FixedLenFeature(
            shape = feature["len"], 
            dtype = feature["type"],
            default_value = [feature["default_value"]] * feature["len"]
        )
    return tf.io.parse_single_example(proto_bytes, feature_description)

In [37]:
parsed_dataset = dataset.map(parse).repeat(5).batch(4096*2, drop_remainder=True)

In [38]:
i = 0
for batch in parsed_dataset:
    i += 1
i

KeyboardInterrupt: 

In [31]:
batch_size = 4096

In [41]:
for batch in parsed_dataset.batch(batch_size, drop_remainder = True):
    print(batch["ad_coec"])

DataLossError: corrupted record at 0